In [1]:
!pip install lifelines
!pip install sklearn
import numpy as np
import pandas as pd
import tensorflow as tf
from sklearn.model_selection import KFold
from tensorflow.keras import layers, models
from tensorflow.keras.regularizers import l1_l2
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, roc_auc_score
from google.colab import drive
# Mount Google Drive

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 349.3/349.3 kB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.5/94.5 kB 6.9 MB/s eta 0:00:00
  Created wheel for autograd-gamma: filename=autograd_gamma-0.5.0-py3-none-any.whl size=4031 sha256=e50347f268e13a3445b540cae9d398a714e5fd0ce112f28aea8776d0d0b99b6b
  Stored in directory: /root/.cache/pip/wheels/25/cc/e0/ef2969164144c899fedb22b338f6703e2b9cf46eeebf254991
Successfully built autograd-gamma
  error: subprocess-exited-with-error
  
  × python setup.py egg_info did not run successfully.
  │ exit code: 1
  ╰─> See above for output.
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  Preparing metadata (setup.py) ... error
error: metadata-generation-failed

× Encountered error while generating package metadata.
╰─> See above for output.

note: This is an issue with the package mentioned above, not pip.
hint: See above for details.


In [52]:
drive.mount('/content/drive')
# train data with DssTime
baseline_data_path = '/content/drive/MyDrive/train_data.csv'
# test data
test_data_path = '/content/drive/MyDrive/test_data.csv'
# train data labeled + unlabeled
train_data_path = '/content/drive/MyDrive/pseudo_labelled_data_vime.csv'

train_data = pd.read_csv(train_data_path)
test_data = pd.read_csv(test_data_path)
baseline_data = pd.read_csv(baseline_data_path)


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [12]:
train_data.columns

Index(['ESR1', 'PGR', 'ERBB2', 'MKI67', 'PLAU', 'ELAVL1', 'EGFR', 'BTRC',
       'FBXO6', 'SHMT2', 'KRAS', 'SRPK2', 'YWHAQ', 'PDHA1', 'EWSR1', 'ZDHHC17',
       'ENO1', 'DBN1', 'PLK1', 'GSK3B', 'Age', 'Size', 'Label',
       'Menopausal State', 'Radio Therapy', 'Chemotherapy', 'Hormone Therapy',
       'Neoplasm Histologic Grade', 'Cellularity', 'Surgery-breast conserving',
       'Surgery-mastectomy', 'Confidence'],
      dtype='object')

In [13]:
test_data.columns

Index(['ESR1', 'PGR', 'ERBB2', 'MKI67', 'PLAU', 'ELAVL1', 'EGFR', 'BTRC',
       'FBXO6', 'SHMT2', 'KRAS', 'SRPK2', 'YWHAQ', 'PDHA1', 'EWSR1', 'ZDHHC17',
       'ENO1', 'DBN1', 'PLK1', 'GSK3B', 'Age', 'Menopausal State', 'Size',
       'Radio Therapy', 'Chemotherapy', 'Hormone Therapy',
       'Neoplasm Histologic Grade', 'Cellularity', 'Surgery-breast conserving',
       'Surgery-mastectomy', 'Label', 'DssTime', 'Event'],
      dtype='object')

In [53]:

# Preprocessing
#X_train_Cindex = train_data.drop(columns=['Label'])#for evaluating the c-index

X_train = train_data.drop(columns=['Label'])  # Include all features except target and event

y_train = train_data['Label']  # Target column

#X_test_Cindex = test_data.drop(columns=['Label'])#for evaluating the c-index

X_test = test_data.drop(columns=['Label','DssTime', 'Event'])  # Include all features except target and event
X_test['Confidence'] = 1.0
y_test = test_data['Label']  # Target column

X_train = X_train[X_test.columns]

# Scale Data
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)



In [54]:
X_train

,ESR1,PGR,ERBB2,MKI67,PLAU,ELAVL1,EGFR,BTRC,FBXO6,SHMT2,...,Menopausal State,Size,Radio Therapy,Chemotherapy,Hormone Therapy,Neoplasm Histologic Grade,Cellularity,Surgery-breast conserving,Surgery-mastectomy,Confidence
0,10.041281,7.376123,9.725825,5.427919,9.300307,6.219375,6.125355,5.888779,7.893369,9.007343,...,0,15.0,0,1,1,2.0,1.0,0.0,1.0,1.000000
1,11.276581,7.331223,9.956267,5.629876,8.119906,5.665620,5.775809,6.251167,8.242063,10.871432,...,1,40.0,1,1,1,3.0,1.0,0.0,1.0,1.000000
2,7.536847,5.587666,11.514514,5.722951,6.741081,6.321480,5.466188,6.956486,7.673015,9.837096,...,1,28.0,0,0,0,2.0,1.0,0.0,1.0,1.000000
3,10.395644,6.531288,9.075396,5.440774,7.861422,5.973844,5.757120,6.026611,7.666777,9.455256,...,1,13.0,1,0,1,2.0,0.5,1.0,0.0,1.000000
4,6.204958,5.172111,8.881671,5.861609,8.530361,6.671294,11.724683,6.046692,7.401715,10.481299,...,1,39.0,1,1,0,3.0,0.0,0.0,1.0,1.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1628,11.628490,5.570690,10.475695,6.032211,9.944405,5.865408,5.703147,6.649948,7.272166,9.750208,...,1,25.0,0,0,1,3.0,1.0,0.0,1.0,0.546133
1629,10.879891,6.431113,10.219154,5.435795,9.224122,5.699195,5.825643,6.404899,7.385644,9.271953,...,1,45.0,0,0,1,3.0,1.0,0.0,1.0,0.544064
1630,9.591235,7.984515,9.935179,5.605596,9.799519,5.808704,5.905282,6.491419,7.865526,9.741103,...,0,25.0,1,0,1,3.0,1.0,1.0,0.0,0.557837
1631,11.055114,8.282737,9.892589,5.753274,8.687667,5.475813,5.587906,6.830579,8.468221,9.482622,...,1,25.0,0,0,1,2.0,0.5,0.0,1.0,0.583897


In [33]:
file_path = '/content/drive/My Drive/DS_GS_Results.xlsx'

In [16]:
from tensorflow.keras import Model
from tensorflow.keras import layers
from tensorflow.keras.regularizers import l1_l2

# Define Custom Loss Function: Negative Partial Log Likelihood
def cox_ph_loss(event, log_hazard_ratio):
    """
    event: Binary censoring indicator (1 if event occurred, 0 if censored)
    log_hazard_ratio: Predicted log hazard ratios
    """
    hazard_ratio = tf.exp(log_hazard_ratio)
    log_cumulative_hazard = tf.math.log(tf.cumsum(hazard_ratio, axis=0, reverse=True))
    uncensored_likelihood = log_hazard_ratio - log_cumulative_hazard
    censored_likelihood = uncensored_likelihood * event
    return -tf.reduce_mean(censored_likelihood)

class DeepSurv(Model):
    def __init__(self, input_dim, hidden_layers, learning_rate,
                 dropout_rate, l1_reg, l2_reg, batch_norm, activation_function,**kwargs):
        # Pass any additional arguments (like `trainable`, `dtype`) to the superclass
        super(DeepSurv, self).__init__(**kwargs)
        self.input_dim = input_dim
        self.hidden_layers_config = hidden_layers
        self.learning_rate = learning_rate
        self.dropout_rate = dropout_rate
        self.l1_reg = l1_reg
        self.l2_reg = l2_reg
        self.batch_norm = batch_norm
        self.activation_function = activation_function

        # Define the layers
        self.hidden_layers = []
        for units in hidden_layers:
            self.hidden_layers.append(layers.Dense(
                units, activation=activation_function,
                kernel_regularizer=l1_l2(l1=l1_reg, l2=l2_reg)
            ))
            if batch_norm:
                self.hidden_layers.append(layers.BatchNormalization())
            self.hidden_layers.append(layers.Dropout(dropout_rate))
        self.output_layer = layers.Dense(1, activation=None)

    def call(self, inputs, training=False):
        x = inputs
        for layer in self.hidden_layers:
            x = layer(x, training=training)
        return self.output_layer(x)

    def get_config(self):
        # Return a dictionary containing all initialization arguments
        config = super(DeepSurv, self).get_config()
        config.update({
            "input_dim": self.input_dim,
            "hidden_layers": self.hidden_layers_config,
            "learning_rate": self.learning_rate,
            "dropout_rate": self.dropout_rate,
            "l1_reg": self.l1_reg,
            "l2_reg": self.l2_reg,
            "batch_norm": self.batch_norm,
        })
        return config

    @classmethod
    def from_config(cls, config):
        # Create a new instance from the configuration
        return cls(**config)




In [17]:
from openpyxl import load_workbook
import pandas as pd

def append_to_excel(file_path, df, sheet_name="Sheet1"):
    """
    Append a DataFrame to an existing Excel sheet or create a new file if it doesn't exist.

    Args:
        file_path (str): Path to the Excel file.
        df (pd.DataFrame): DataFrame to append.
        sheet_name (str): Sheet name to write to. Defaults to "Sheet1".
    """
    try:
        # Load the existing workbook
        book = load_workbook(file_path)

        # Check if the sheet exists
        if sheet_name in book.sheetnames:
            sheet = book[sheet_name]
            startrow = sheet.max_row  # Get the last row in the existing sheet
        else:
            startrow = 0  # Write from the top

        # Create an Excel writer using openpyxl engine
        with pd.ExcelWriter(file_path, engine='openpyxl', mode='a', if_sheet_exists='overlay') as writer:
            # Write data to the sheet
            df.to_excel(writer, sheet_name=sheet_name, index=False, startrow=startrow, header=startrow == 0)

        print(f"Data appended successfully to {file_path} in sheet {sheet_name}.")
    except FileNotFoundError:
        # If the file does not exist, create it
        print(f"File not found. Creating a new file: {file_path}")
        df.to_excel(file_path, sheet_name=sheet_name, index=False)
        print(f"New file created successfully with data in sheet {sheet_name}.")
    except Exception as e:
        print(f"An error occurred: {e}")


In [57]:
input_dim = X_train_scaled.shape[1]
hidden_layers = [[64, 128, 64]]  # Architecture
learning_rate = 1e-3
dropout_rate = 0.2
l1_reg = 1e-5
l2_reg = 1e-3
activation_function ='relu'
batch_norm = False
#batch_norm
epochs = 50
# Define the time points for which we want survival probabilities
time_points = [1, 10, 20, 30, 40, 50, 60, 70, 80, 90, 100]

num_of_models = len(hidden_layers)
print("number of models: " + str(num_of_models))

number of models: 2


## Cross Validation

## Run it once

In [ ]:
results = []
current_mode_nr = 1
kf = KFold(n_splits=5, shuffle=True, random_state=42)  # 5-fold cross-validation

for hidden_layer in hidden_layers:
            fold_accuracies = []
            fold_roc_aucs = []

            for train_idx, val_idx in kf.split(X_train_scaled):
                # Split the data into training and validation sets
                X_train_fold, X_val_fold = X_train_scaled[train_idx], X_train_scaled[val_idx]
                y_train_fold, y_val_fold = y_train[train_idx], y_train[val_idx]

                # Define the model
                deep_surv = DeepSurv(
                    input_dim=input_dim,
                    hidden_layers=hidden_layer,
                    learning_rate=learning_rate,
                    dropout_rate=dropout_rate,
                    l1_reg=l1_reg,
                    l2_reg=l2_reg,
                    batch_norm=batch_norm,
                    activation_function=activation_function
                )

                # Compile the model
                optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate)
                deep_surv.compile(optimizer=optimizer, loss=cox_ph_loss)

                # Train the model
                deep_surv.fit(
                    X_train_fold,
                    y_train_fold,
                    batch_size=32,
                    epochs=epochs,
                    verbose=0
                )

                # Predict Risk Scores for Training Data
                risk_scores_train = deep_surv.predict(X_train_scaled).flatten()
                # Step 2: Compute and Simplify the Baseline Hazard Function
                # ---------------------------------------------------------
                # Sort data by survival times
                # Align the predicted risk scores with the sorted survival times.
                # Sorting ensures that the baseline hazard calculation respects the temporal order.
                survival_times = baseline_data['DssTime']  # Use the correct survival time column
                sorted_indices = np.argsort(survival_times)  # returns sorted indices of survival times
                sorted_times = survival_times.iloc[sorted_indices]  # uses sorted indices to rearrange survival times
                sorted_risks = np.exp(risk_scores_train[sorted_indices])  # convert the log risk scores into risk scores using the exponential function

                # Calculate Cumulative Baseline Hazard
                unique_times, counts = np.unique(sorted_times, return_counts=True)  # Unique times and their counts of event for that time
                cumulative_baseline_hazard = []
                risk_sum = np.sum(sorted_risks)

                for time, event_count in zip(unique_times, counts):
                    hazard = event_count / risk_sum  # Events / At-risk in Formula dN(t)/R(t)
                    if cumulative_baseline_hazard:
                        cumulative_baseline_hazard.append(cumulative_baseline_hazard[-1] + hazard)  # Add to cumulative
                    else:
                        cumulative_baseline_hazard.append(hazard)  # First time point
                    risk_sum -= np.sum(sorted_risks[sorted_times == time])  # Subtract patients who are no longer at risk

                # Clip the cumulative baseline hazard to prevent extreme values
                cumulative_baseline_hazard = np.clip(cumulative_baseline_hazard, 1e-6, None)

                # Estimate the cumulative baseline hazard for specific time points using interpolation.
                # Interpolate baseline hazard for specified time points
                baseline_hazards = np.interp(
                    time_points,
                    unique_times,
                    cumulative_baseline_hazard,
                    left=cumulative_baseline_hazard[0],
                    right=cumulative_baseline_hazard[-1]
                )


                risk_scores_val = deep_surv.predict(X_val_fold).flatten()

                # Evaluate predictions
                survival_probabilities = {}
                for t, baseline_hazard in zip(time_points, baseline_hazards):
                    risk_factors = np.exp(risk_scores_val)
                    risk_factors = np.clip(risk_factors, 1e-3, 1e3)
                    survival_probabilities[t] = np.exp(-baseline_hazard * risk_factors)

                t60_probabilities = survival_probabilities[60]
                predicted_labels_t60 = (t60_probabilities <= 0.5).astype(int)

                accuracy = accuracy_score(y_val_fold, predicted_labels_t60)
                roc_auc = roc_auc_score(y_val_fold, predicted_labels_t60)

                # Record fold results
                fold_accuracies.append(accuracy)
                fold_roc_aucs.append(roc_auc)

            # Compute average metrics across folds
            avg_accuracy = np.mean(fold_accuracies)
            avg_roc_auc = np.mean(fold_roc_aucs)

            # Store the results
            results.append({
                'Hidden Layers': hidden_layer,
                'Learning Rate': learning_rate,
                'Dropout Rate': dropout_rate,
                'L1 Regularization': l1_reg,
                'L2 Regularization': l2_reg,
                'Average Accuracy': avg_accuracy,
                'Average ROC AUC': avg_roc_auc,
            })

            print(f"Finished model nr: {current_mode_nr} | Avg Accuracy: {avg_accuracy:.4f} | Avg ROC AUC: {avg_roc_auc:.4f}")
            print(hidden_layer)
            print("fold_accuracies",fold_accuracies)
            print("fold_roc_aucs",fold_roc_aucs)
            current_mode_nr += 1

# Convert results to a DataFrame and save to Excel
result_df = pd.DataFrame(results)
append_to_excel('/content/drive/My Drive/DS_GS_Results.xlsx', result_df, sheet_name="Sheet1")

41/41 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step 
41/41 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step 
41/41 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step 
41/41 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step 
41/41 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step 
Finished model nr: 1 | Avg Accuracy: 0.8311 | Avg ROC AUC: 0.7967
[128, 64, 32]
fold_accuracies [0.8521400778210116, 0.7859922178988327, 0.7898832684824902, 0.8754863813229572, 0.8521400778210116]
fold_roc_aucs [0.786858685868587, 0.7506132756132757, 0.7718482028826856, 0.8543544635068335, 0.8200578990901571]
41/41 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step 
41/41 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step 
41/41 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step 
41/41 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/st

In [58]:
results = []
current_mode_nr = 1
for hidden_layer in hidden_layers:

                deep_surv = DeepSurv(
                input_dim=input_dim,
                hidden_layers=hidden_layer,
                learning_rate=learning_rate,
                dropout_rate=dropout_rate,
                l1_reg=l1_reg,
                l2_reg=l2_reg,
                batch_norm=batch_norm,
                activation_function= activation_function
                )

                optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate)
                # Compile the model with the optimizer and loss function
                deep_surv.compile(optimizer=optimizer, loss=cox_ph_loss)

                # Train the model on the training data
                deep_surv.fit(
                    X_train_scaled,
                    y_train,
                    batch_size=32,
                    epochs=epochs,
                    verbose=0,
                )

                # Predict Risk Scores for Training Data
                risk_scores_train = deep_surv.predict(X_train_scaled).flatten()
                # Step 2: Compute and Simplify the Baseline Hazard Function
                # ---------------------------------------------------------
                # Sort data by survival times
                # Align the predicted risk scores with the sorted survival times.
                # Sorting ensures that the baseline hazard calculation respects the temporal order.
                survival_times = baseline_data['DssTime']  # Use the correct survival time column
                sorted_indices = np.argsort(survival_times)  # returns sorted indices of survival times
                sorted_times = survival_times.iloc[sorted_indices]  # uses sorted indices to rearrange survival times
                sorted_risks = np.exp(risk_scores_train[sorted_indices])  # convert the log risk scores into risk scores using the exponential function

                # Calculate Cumulative Baseline Hazard
                unique_times, counts = np.unique(sorted_times, return_counts=True)  # Unique times and their counts of event for that time
                cumulative_baseline_hazard = []
                risk_sum = np.sum(sorted_risks)

                for time, event_count in zip(unique_times, counts):
                    hazard = event_count / risk_sum  # Events / At-risk in Formula dN(t)/R(t)
                    if cumulative_baseline_hazard:
                        cumulative_baseline_hazard.append(cumulative_baseline_hazard[-1] + hazard)  # Add to cumulative
                    else:
                        cumulative_baseline_hazard.append(hazard)  # First time point
                    risk_sum -= np.sum(sorted_risks[sorted_times == time])  # Subtract patients who are no longer at risk

                # Clip the cumulative baseline hazard to prevent extreme values
                cumulative_baseline_hazard = np.clip(cumulative_baseline_hazard, 1e-6, None)

                # Estimate the cumulative baseline hazard for specific time points using interpolation.
                # Interpolate baseline hazard for specified time points
                baseline_hazards = np.interp(
                    time_points,
                    unique_times,
                    cumulative_baseline_hazard,
                    left=cumulative_baseline_hazard[0],
                    right=cumulative_baseline_hazard[-1]
                )

                # Predict Risk Scores for Testing Data
                risk_scores_test = deep_surv.predict(X_test_scaled).flatten()

                # Calculate survival probabilities
                # Compute survival probabilities for all test patients
                survival_probabilities = {}
                for t, baseline_hazard in zip(time_points, baseline_hazards):
                    # Correctly compute risk factors for each patient
                    risk_factors = np.exp(risk_scores_test)
                    risk_factors = np.clip(np.exp(risk_scores_test), 1e-3, 1e3)  # Adjust range to [1e-3, 1e3]
                    survival_probabilities[t] = np.exp(-baseline_hazard * risk_factors)

                # extract crucial survival time of 60 months
                t60_probabilities_baseline = survival_probabilities[60]
                # Predict labels based on threshold
                optimal_threshold = 0.50  # Adjust threshold as necessary
                # label test data
                predicted_labels_t60_baseline = (t60_probabilities_baseline <= optimal_threshold).astype(int)

                # Evaluate Model for t=60
                # ------------------------
                accuracy_baseline = accuracy_score(y_test, predicted_labels_t60_baseline)
                roc_auc_baseline = roc_auc_score(y_test, predicted_labels_t60_baseline)

                # Store the results
                results.append({
                        'Hidden Layers': hidden_layer,
                        'Learning Rate': learning_rate,
                        'Dropout Rate': dropout_rate,
                        'L1 Regularization': l1_reg,
                        'L2 Regularization': l2_reg,
                        'Accuracy': accuracy_baseline,
                        'ROC AUC': roc_auc_baseline,
                })

                # Convert results to a DataFrame for visualization
                result_df = pd.DataFrame(results)
                print("finished model nr: ", current_mode_nr)
                current_mode_nr = 1+current_mode_nr

# save to excel
append_to_excel('/content/drive/My Drive/DS_GS_Results.xlsx', result_df, sheet_name="Sheet1")

52/52 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step 
finished model nr:  1
52/52 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step 
finished model nr:  2
Data appended successfully to /content/drive/My Drive/DS_GS_Results.xlsx in sheet Sheet1.


In [59]:
result_df

,Hidden Layers,Learning Rate,Dropout Rate,L1 Regularization,L2 Regularization,Accuracy,ROC AUC
0,"[128, 64, 32]",0.001,0.2,0.00001,0.001,0.726496,0.718328
1,"[256, 64, 64, 32]",0.001,0.2,0.00001,0.001,0.726496,0.719355


In [ ]:
result_df

""
